In [1]:
#Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

In [ ]:
## Reading the Data
airline_df = pd.read_csv(r"C:\Users\magak\Desktop\Projects\Airline Analysis\Data\Airline Dataset.csv")
airline_df.head()

In [3]:
#Features and target variable

X = airline_df.drop(columns=['coach_price', 'firstclass_price'])
y = airline_df['coach_price']

# Define the preprocessing steps for numerical and categorical features
numerical_features = ['miles', 'passengers', 'delay', 'hours']
categorical_features = ['inflight_meal', 'inflight_entertainment', 'inflight_wifi', 'day_of_week', 'redeye', 'weekend']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_model)
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [100],
    'model__max_depth': [None],
    'model__min_samples_split': [2]
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

Best parameters found:  {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Best cross-validation score:  1032.0059625754432


In [4]:
# Evaluate the model on the test set
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error on test set: ", mse)
print("R^2 Score on test set: ", r2)

Mean Squared Error on test set:  1038.5994245187446
R^2 Score on test set:  0.7751163082766238
